In [2]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.formula.api as smf
from regtabletotext import prettify_result

First, I'll load in the data I have stored in my DB, this includes Fama's precomputed values, CRSP, and Compustat data.

In [3]:
crspdb = sqlite3.connect(
  database="../data/crsp.db.sqlite"
)

compustatdb = sqlite3.connect(
    database="../data/compustat.db.sqlite"
)

db = sqlite3.connect(
    database="../data/db.sqlite"
)

crsp_monthly = (pd.read_sql_query(
    sql=("SELECT permno, gvkey, date, ret_excess, mktcap, "
         "mktcap_lag, exchange FROM crsp_monthly"),
    con=crspdb,
    parse_dates={"date"})
  .dropna()
)

compustat = (pd.read_sql_query(
    sql="SELECT gvkey, datadate, be, op, inv FROM compustat",
    con=compustatdb,
    parse_dates={"datadate"})
  .dropna()
)

factors_ff3_monthly = pd.read_sql_query(
  sql="SELECT date, smb, hml FROM factors_ff3_monthly",
  con=db,
  parse_dates={"date"}
)

factors_ff5_monthly = pd.read_sql_query(
  sql="SELECT date, smb, hml, rmw, cma FROM factors_ff5_monthly",
  con=db,
  parse_dates={"date"}
)

Now I will get the data I need. For each firm, the size value is from June, the BE/ME is calculated from the year before and BE is from CRSP, while the ME is from Compustat.

In [4]:
size = (crsp_monthly
  .query("date.dt.month == 6")
  .assign(sorting_date=lambda x: (x["date"]+pd.DateOffset(months=1)))
  .get(["permno", "exchange", "sorting_date", "mktcap"])
  .rename(columns={"mktcap": "size"})
)

market_equity = (crsp_monthly
  .query("date.dt.month == 12")
  .assign(sorting_date=lambda x: (x["date"]+pd.DateOffset(months=7)))
  .get(["permno", "gvkey", "sorting_date", "mktcap"])
  .rename(columns={"mktcap": "me"})
)

book_to_market = (compustat
  .assign(
    sorting_date=lambda x: (pd.to_datetime(
      (x["datadate"].dt.year+1).astype(str)+"0701", format="%Y%m%d")
    )
  )
  .merge(market_equity, how="inner", on=["gvkey", "sorting_date"])
  .assign(bm=lambda x: x["be"]/x["me"])
  .get(["permno", "sorting_date", "me", "bm"])
)

sorting_variables = (size
  .merge(book_to_market, how="inner", on=["permno", "sorting_date"])
  .dropna()
  .drop_duplicates(subset=["permno", "sorting_date"])
 )

After getting all the variables, we can assign each firm at a specific time into their portfilios based on the two factors. I follow the 2x3 factoring model here.

In [5]:
def assign_portfolio(data, sorting_variable, percentiles):
    """Assign portfolios to a bin according to a sorting variable."""
    
    breakpoints = (data
      .query("exchange == 'NYSE'")
      .get(sorting_variable)
      .quantile(percentiles, interpolation="linear")
    )
    breakpoints.iloc[0] = -np.Inf
    breakpoints.iloc[breakpoints.size-1] = np.Inf
    
    assigned_portfolios = pd.cut(
      data[sorting_variable],
      bins=breakpoints,
      labels=pd.Series(range(1, breakpoints.size)),
      include_lowest=True,
      right=False
    )
    
    return assigned_portfolios

portfolios = (sorting_variables
  .groupby("sorting_date")
  .apply(lambda x: x
    .assign(
      portfolio_size=assign_portfolio(x, "size", [0, 0.5, 1]),
      portfolio_bm=assign_portfolio(x, "bm", [0, 0.3, 0.7, 1])
    )
  )
  .reset_index(drop=True)
  .get(["permno", "sorting_date", "portfolio_size", "portfolio_bm"])
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\2271133125.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x


After getting the portfolios,we need to calculate the weighted yearly return for each firm. We can do this by taking the excess returns from the last year from June.
I use the weighted mean of the monthly returns for each firm based on the market cap from the previous month. Another possiblity is using the geometric mean, but there are a lot of numbers, so I opted against this.

In [6]:
portfolios = (crsp_monthly
  .assign(
    sorting_date=lambda x: (pd.to_datetime(
      x["date"].apply(lambda x: str(x.year-1)+
        "0701" if x.month <= 6 else str(x.year)+"0701")))
  )
  .merge(portfolios, how="inner", on=["permno", "sorting_date"])
)

In [22]:
portfolios

,permno,gvkey,date,ret_excess,mktcap,mktcap_lag,exchange,sorting_date,portfolio_size,portfolio_bm,portfolio_op,portfolio_inv
0,10001,012994,1988-07-01,0.024900,6.38600,6.20000,NASDAQ,1988-07-01,1,3,1,1
1,10001,012994,1988-08-01,0.023226,6.57200,6.38600,NASDAQ,1988-07-01,1,3,1,1
2,10001,012994,1988-09-01,-0.027959,6.36225,6.57200,NASDAQ,1988-07-01,1,3,1,1
3,10001,012994,1988-10-01,0.033116,6.61175,6.36225,NASDAQ,1988-07-01,1,3,1,1
4,10001,012994,1988-11-01,-0.005700,6.61175,6.61175,NASDAQ,1988-07-01,1,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2590791,93374,184899,2023-08-01,-0.031322,6358.34448,6533.59068,NYSE,2023-07-01,2,3,1,1
2590792,93374,184899,2023-09-01,-0.080483,5823.32814,6358.34448,NYSE,2023-07-01,2,3,1,1
2590793,93374,184899,2023-10-01,-0.094096,5307.63064,5823.32814,NYSE,2023-07-01,2,3,1,1
2590794,93374,184899,2023-11-01,0.154231,6149.58760,5307.63064,NYSE,2023-07-01,2,3,1,1


In [7]:
factors_replicated = (portfolios
  .groupby(["portfolio_size", "portfolio_bm", "date"])
  .apply(lambda x: pd.Series({
    "ret": np.average(x["ret_excess"], weights=x["mktcap_lag"])
    })
   )
  .reset_index()
  .groupby("date")
  .apply(lambda x: pd.Series({
    "smb_replicated": (
      x["ret"][x["portfolio_size"] == 1].mean() - 
        x["ret"][x["portfolio_size"] == 2].mean()),
    "hml_replicated": (
      x["ret"][x["portfolio_bm"] == 3].mean() -
        x["ret"][x["portfolio_bm"] == 1].mean())
    }))
  .reset_index()
)

factors_replicated = (factors_replicated
  .merge(factors_ff3_monthly, how="inner", on="date")
  .round(4)
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\83293942.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["portfolio_size", "portfolio_bm", "date"])
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\83293942.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\83293942.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the 

In [8]:
model_smb = (smf.ols(
    formula="smb ~ smb_replicated", 
    data=factors_replicated
  )
  .fit()
)
prettify_result(model_smb)

OLS Model:
smb ~ smb_replicated

Coefficients:
                Estimate  Std. Error  t-Statistic  p-Value
Intercept         -0.000       0.000       -1.408     0.16
smb_replicated     0.989       0.004      234.132     0.00

Summary statistics:
- Number of observations: 738
- R-squared: 0.987, Adjusted R-squared: 0.987
- F-statistic: 54,817.973 on 1 and 736 DF, p-value: 0.000



The above is the comparison between my SMB results and Fama and French's results. I have a correlation of 99%, which is pretty close.

In [9]:
model_hml = (smf.ols(
    formula="hml ~ hml_replicated", 
    data=factors_replicated
  )
  .fit()
)
prettify_result(model_hml)

OLS Model:
hml ~ hml_replicated

Coefficients:
                Estimate  Std. Error  t-Statistic  p-Value
Intercept          0.000       0.000        1.873    0.061
hml_replicated     0.963       0.007      135.623    0.000

Summary statistics:
- Number of observations: 738
- R-squared: 0.962, Adjusted R-squared: 0.961
- F-statistic: 18,393.559 on 1 and 736 DF, p-value: 0.000



My high minus low is also very close, with a R^2 of 96%

Now we can also include the other two factors in the Fama-French 5 Factor Model. We can follow a similar process as the other two variables. The only difference is that we need to compute smb based on all of the other variables together, whereas the other factors is only dependent on size. This is because Fama had a 2x5 factor model.

In [10]:
other_sorting_variables = (compustat
  .assign(
    sorting_date=lambda x: (pd.to_datetime(
      (x["datadate"].dt.year+1).astype(str)+"0701", format="%Y%m%d")
    )
  )
  .merge(market_equity, how="inner", on=["gvkey", "sorting_date"])
  .assign(bm=lambda x: x["be"]/x["me"])
  .get(["permno", "sorting_date", "me", "bm", "op", "inv"])
)

sorting_variables = (size
  .merge(other_sorting_variables, how="inner", on=["permno", "sorting_date"])
  .dropna()
  .drop_duplicates(subset=["permno", "sorting_date"])
 )

In [11]:
portfolios = (sorting_variables
  .groupby("sorting_date")
  .apply(lambda x: x
    .assign(
      portfolio_size=assign_portfolio(x, "size", [0, 0.5, 1])
    )
  )
  .reset_index(drop=True)
  .groupby(["sorting_date", "portfolio_size"])
  .apply(lambda x: x
    .assign(
      portfolio_bm=assign_portfolio(x, "bm", [0, 0.3, 0.7, 1]),
      portfolio_op=assign_portfolio(x, "op", [0, 0.3, 0.7, 1]),
      portfolio_inv=assign_portfolio(x, "inv", [0, 0.3, 0.7, 1])
    )
  )
  .reset_index(drop=True)
  .get(["permno", "sorting_date", 
        "portfolio_size", "portfolio_bm",
        "portfolio_op", "portfolio_inv"])
)

portfolios = (crsp_monthly
  .assign(
    sorting_date=lambda x: (pd.to_datetime(
      x["date"].apply(lambda x: str(x.year-1)+
        "0701" if x.month <= 6 else str(x.year)+"0701")))
  )
  .merge(portfolios, how="inner", on=["permno", "sorting_date"])
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\2825916934.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\2825916934.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["sorting_date", "portfolio_size"])
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\2825916934.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping co

In [12]:
portfolios_value = (portfolios
  .groupby(["portfolio_size", "portfolio_bm", "date"])
  .apply(lambda x: pd.Series({
      "ret": np.average(x["ret_excess"], weights=x["mktcap_lag"])
    })
  )
  .reset_index()
)

factors_value = (portfolios_value
  .groupby("date")
  .apply(lambda x: pd.Series({
    "hml_replicated": (
      x["ret"][x["portfolio_bm"] == 3].mean() - 
        x["ret"][x["portfolio_bm"] == 1].mean())})
  )
  .reset_index()
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\1393307411.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["portfolio_size", "portfolio_bm", "date"])
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\1393307411.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\1393307411.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pand

In [13]:
portfolios_profitability = (portfolios
  .groupby(["portfolio_size", "portfolio_op", "date"])
  .apply(lambda x: pd.Series({
      "ret": np.average(x["ret_excess"], weights=x["mktcap_lag"])
    })
  )
  .reset_index()
)

factors_profitability = (portfolios_profitability
  .groupby("date")
  .apply(lambda x: pd.Series({
    "rmw_replicated": (
      x["ret"][x["portfolio_op"] == 3].mean() - 
        x["ret"][x["portfolio_op"] == 1].mean())})
  )
  .reset_index()
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\1016769137.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["portfolio_size", "portfolio_op", "date"])
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\1016769137.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\1016769137.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pand

In [14]:
portfolios_investment = (portfolios
  .groupby(["portfolio_size", "portfolio_inv", "date"])
  .apply(lambda x: pd.Series({
      "ret": np.average(x["ret_excess"], weights=x["mktcap_lag"])
    })
  )
  .reset_index()
)

factors_investment = (portfolios_investment
  .groupby("date")
  .apply(lambda x: pd.Series({
    "cma_replicated": (
      x["ret"][x["portfolio_inv"] == 1].mean() - 
        x["ret"][x["portfolio_inv"] == 3].mean())})
  )
  .reset_index()
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\468946624.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["portfolio_size", "portfolio_inv", "date"])
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\468946624.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\468946624.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas

In [15]:
factors_size = (
  pd.concat(
    [portfolios_value, portfolios_profitability, portfolios_investment], 
    ignore_index=True
  )
  .groupby("date")
  .apply(lambda x: pd.Series({
    "smb_replicated": (
      x["ret"][x["portfolio_size"] == 1].mean() - 
        x["ret"][x["portfolio_size"] == 2].mean())})
  )
  .reset_index()
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25856\426887677.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [16]:
factors_replicated = (factors_size
  .merge(factors_value, how="outer", on="date")
  .merge(factors_profitability, how="outer", on="date")
  .merge(factors_investment, how="outer", on="date")
)

factors_replicated = (factors_replicated
  .merge(factors_ff5_monthly, how="inner", on="date")
  .round(4)
)

In [17]:
model_smb = (smf.ols(
    formula="smb ~ smb_replicated", 
    data=factors_replicated
  )
  .fit()
)
prettify_result(model_smb)

OLS Model:
smb ~ smb_replicated

Coefficients:
                Estimate  Std. Error  t-Statistic  p-Value
Intercept         -0.000       0.000       -1.450    0.148
smb_replicated     0.964       0.004      227.024    0.000

Summary statistics:
- Number of observations: 726
- R-squared: 0.986, Adjusted R-squared: 0.986
- F-statistic: 51,540.074 on 1 and 724 DF, p-value: 0.000



In [18]:
model_hml = (smf.ols(
    formula="hml ~ hml_replicated", 
    data=factors_replicated
  )
  .fit()
)
prettify_result(model_hml)

OLS Model:
hml ~ hml_replicated

Coefficients:
                Estimate  Std. Error  t-Statistic  p-Value
Intercept          0.001        0.00        1.958    0.051
hml_replicated     0.988        0.01       98.140    0.000

Summary statistics:
- Number of observations: 726
- R-squared: 0.930, Adjusted R-squared: 0.930
- F-statistic: 9,631.494 on 1 and 724 DF, p-value: 0.000



In [19]:
model_rmw = (smf.ols(
    formula="rmw ~ rmw_replicated", 
    data=factors_replicated
  )
  .fit()
)
prettify_result(model_rmw)

OLS Model:
rmw ~ rmw_replicated

Coefficients:
                Estimate  Std. Error  t-Statistic  p-Value
Intercept           0.00       0.000        0.183    0.855
rmw_replicated      0.95       0.009      107.455    0.000

Summary statistics:
- Number of observations: 726
- R-squared: 0.941, Adjusted R-squared: 0.941
- F-statistic: 11,546.613 on 1 and 724 DF, p-value: 0.000



In [20]:
model_cma = (smf.ols(
    formula="cma ~ cma_replicated", 
    data=factors_replicated
  )
  .fit()
)
prettify_result(model_cma)

OLS Model:
cma ~ cma_replicated

Coefficients:
                Estimate  Std. Error  t-Statistic  p-Value
Intercept          0.001       0.000        3.845      0.0
cma_replicated     0.964       0.008      121.120      0.0

Summary statistics:
- Number of observations: 726
- R-squared: 0.953, Adjusted R-squared: 0.953
- F-statistic: 14,670.112 on 1 and 724 DF, p-value: 0.000

